<a href="https://colab.research.google.com/github/jackjameswillis/BSc-Final-Year-Project/blob/main/AdaNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First we install adanet

In [1]:
!pip install adanet

We now import libraries

In [2]:
import adanet
import tensorflow as tf
import numpy as np
import keras

AdaNet uses the Tensorflow Estimator library, so we will also use this library. Before we implement estimators, however, we need to acquire the data. The dataset used in this project is MNIST, a dataset of handwritten digits. For more information on this dataset see the dedicated notebook.

We can easily acquire the dataset through Keras, which provides us with the data in numpy data structures.

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='mnist.npz')

Before continuing we set a seed for random number generators.

In [4]:
seed = 201338

The Tensorflow Estimator system uses a data input convention where we provide an input function, returning features and labels. For this AdaNet implementation we need not micro-manage the order of data, so we can be liberal with the specifics input generator.

In [5]:
# The key for the input features. We have one set of inputs,
# these being the flattened images, so we provide one key.
feat_key = "x"

# We use this generator function to produce the dataset later.
def generator(X, Y):

  def _gen():

    for x, y in zip(X, Y):

      yield tf.reshape(x, [len(x)]), y

  return _gen

# We map this function onto elements of the database 
# to make it compatible with the expected format of 
# input data.
def featurise(x, y):
  
  features = {feat_key: x}

  return features, y

# This function returns an input function compatible with
# the training functions used later. It manages data for
# both train time and test time, which are provided
# as seperate datasets when downloaded.
def input_fn(partition, batch_size, repeat=False):

  if partition == 'train':

    x, y = (x_train, y_train)
    
  else:

    x, y = (x_test, y_test)

  # We take whichever of training and testing data
  # and first randomize the their order.
  rng = np.random.default_rng(seed=seed)

  rng.shuffle(x)

  rng = np.random.default_rng(seed=seed)

  rng.shuffle(y)

  # We apply preprocessing to this data, which merely
  # squishes the greyscale values between 0 and 1 and
  # flattens the images.
  _x = []

  for i in range(len(x)):
    
    _x.append(x[i]/255)
    
    _x[i] = _x[i].flatten()
  
  x = _x

  # This is the function returned, which generates the 
  # dataset from the preprocessed data.
  def _input_fn():

    # Use our generator function to produce the dataset
    # with the appropriate data sizes and data types.
    dataset = tf.data.Dataset.from_generator(generator(x, y), (tf.float32, tf.int32), ((len(x[0])), ()))

    # If we are training, we want training to continue
    # after we reach the final batch of the dataset. We
    # achieve this by 'repeating' the dataset.
    if repeat: dataset = dataset.repeat()

    # Apply featre function to data and batch according
    # to batch_size parameter.
    dataset = dataset.map(featurise).batch(batch_size)

    # Return the dataset ready to train on.
    return dataset

  return _input_fn

In accordance with the convention, we initialise some of the required systems for using Tensorflow Estimators.

First, we recognise that all sub-estimators will have the same output size, sub-estimators will have the same loss function, etc. Therefore, we define an estimator head that can be applied to all sub-estimators. The ensembling estimator can also use this head.

In [6]:
head = tf.estimator.MultiClassHead(n_classes=10)

Second, all sub-estimators will use the same input features. Therefore we can define feature columns that can be applied to all sub-estimators.

In [7]:
feature_columns = [tf.feature_column.numeric_column(feat_key, shape=[28*28])]

We now define a function that produces a sub-estimator candidate pool. The function returns a dictionary of candidate perceptrons, which is to say that each perceptron has $28 \cdot 28$ inputs and an output layer of $10$ units, which have varying dropout rates. Dropout acts to introduce sparsity into candidate weights.

In [8]:
size = 10

In [9]:
# Function that returns a dictionary pool of candidate
# sub-estimator perceptrons. The size parameter governs
# the number of candidates, where the index of their
# addition to the pool governs their dropout rate.
def candidate_pool(size):

  pool = {}

  for i in range(size):

    # The current candidate's dropout rate is a fraction of the total number of
    # candidates in accordiance with the candidate's index.
    dropout = i/size

    key = 'Dropout' + str(dropout)

    # Add to the pool a perceptron with the desired dropout rate. Note that the
    # DNNEstimator was used here instead of the LinearEstimator simply to control
    # the optimizer, activation function and dropout.
    pool[key] = tf.estimator.DNNEstimator(
        
        head=head,

        hidden_units=[],

        feature_columns=feature_columns,

        # 'SGD' is in reference to stochastic Gradient Descent.
        optimizer='SGD',

        activation_fn=tf.nn.sigmoid,

        dropout=dropout)
  
  return pool

Now we train the model. We begin by initialising the AdaNet model, which ensembles selected candidates using a linear classifier. We train and evaluate the model, providing input with our previously defined input data function.

In [10]:
steps = 5000

batch_size = 64

# Number of candidates in the candidate pool.
pool_size = 10

# Initialise our AdaNet ensemble model.
model = adanet.AutoEnsembleEstimator(
    
    head=head, 
    
    candidate_pool=candidate_pool(pool_size),
        
    max_iteration_steps=steps,

    # We evaluate candidate performance within the AdaNet
    # ensembler using the AdaNet loss function.
    evaluator=adanet.Evaluator(
        
        input_fn=input_fn(
            
            partition='train', 
            
            batch_size=batch_size),

        steps=None))

# Train our AdaNet ensemble.
model.train(
    
    input_fn=input_fn(
        
        partition='train',

        batch_size=batch_size,

        repeat=True),
        
    max_steps=steps)

# Evaluate our AdaNet ensemble
results = model.evaluate(
    
    input_fn=input_fn(
        
        partition='test', 
        
        batch_size=batch_size),

    steps=None)

print("Accuracy:", results["accuracy"])

print("Loss:", results["average_loss"])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9w84rccl', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir'

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpr228rsj1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use output_signature instead


...

INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.899, average_loss = 0.37756506, best_ensemble_index_0 = 0, global_step = 5000, iteration = 0, loss = 0.37644818


INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.899, average_loss = 0.37756506, best_ensemble_index_0 = 0, global_step = 5000, iteration = 0, loss = 0.37644818


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpr228rsj1/model.ckpt-5000


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpr228rsj1/model.ckpt-5000


Accuracy: 0.899
Loss: 0.37756506


We can now look at the ensemble architecture.

In [11]:
architecture = results['architecture/adanet/ensembles']

In [12]:
architecture

b'\n5\n\x13architecture/adanetB\x14\x08\x07\x12\x00B\x0e| Dropout0.0 |J\x08\n\x06\n\x04text'